# Puppet overview 
* State based configuration management tool, it consists of many tools to reinforce that state.
* It's unique from its siblings as it focuses on modelling the infrastructure by specifying the desired state that processes or tasks.
* Adaptable and abstract coding makes it platform agnostic to any operating systems. 
* Abstraction: Module, Profiles (subset of configs), Role (Collection of profile) 
* Reporting capability - inventory audit and complience.
* Self-healing - puppet makes a sanity check in each 30 mins to match the systems state to the expected state, and takes countermeasures if mismatches. 
* Terms
    * __Nodes__ : Individual server or device
    * __Resources__: Individiual units of configuration in puppet (Built-in / custom)
    * __Class__: A collection of puppet code 
    * __Manifest__: A text file for holding puppet code (*.pp), it contains a single class
    * __Profile__: A class that defines a specific set of configuration configs
    * __Role__ : Business role of a node, a role is a collection of profiles 

# Seting up Environment 
## Installing puppet master
* __Step 0: Be the root__ : `sudo su -` (Note: Step 8 will not execute if you use "sudo")
* __Step 1: Install puppet master__ <br>
`rpm -Uvh https://yum.puppet.com/puppet6-release-el-7.noarch.rpm`

* __Step 2: Install puppet server__ <br>
`yum install -y puppetserver vim git `

* __Step 3: Change the resource allocaion.__
    * goto `nano /etc/sysconfig/puppetserver` to alter the configuration.
    * Change the `JAVA_ARGS` parameters to change the memory allocaion
    * `-Xms`and `Xmx` stands for min and max memory allocation. We allocate 512M to 1G<br>
```
# Modify this if you'd like to change the memory allocation, enable JMX, etc
JAVA_ARGS="-Xms512m -Xmx1g -Djruby.logger.class=com.puppetlabs.jruby_utils.jruby.Slf4jLogger"
```

* __Step 4: Start Puppet server__ <br>
    * `systemctl start puppetserver` to start the service 
    * `systemctl enable puppetserver` to put the service in system startup 

* __Step 5: Verify status__ <br>
`systemctl status puppetserver`

* __Step 6: Config the puppet agent to point its master__ <br>
    * goto `/etc/puppetlabs/puppet/puppet.conf`
    * The master section must be there by default, add an Agent section. <br>
    ```
    [agent]
    server = localhost.localdomain #using a selfloop
    ```
* __Step 7: Install Ruby,Gem__ : Recommanded to use the ruby package that comes with puppet. Add the location to system PATH usig 
    * `nano .bash_profile`
    ```    
    # .bash_profile
    # Get the aliases and functions
    if [ -f ~/.bashrc ]; then
        . ~/.bashrc
    fi
    
    # User specific environment and startup programs
    
    #add this line for puppet-ruby integration
    PATH=$PATH:/opt/puppetlabs/puppet/bin 
    
    PATH=$PATH:$HOME/.local/bin:$HOME/bin

    export PATH
    ```
    * `exec bash` to start a new bash session
    * `source .bash_profile` to apply changes 
    * Verify by `$gem` command 
    
* __Step 8: Install r10k__ :  `gem install r10k`
* __Step 9: Verify overall setup__

```
[root@localhost ~]# puppet agent -t 
Info: Using configured environment 'production'
Info: Retrieving pluginfacts
Info: Retrieving plugin
Info: Retrieving locales
Info: Caching catalog for localhost.localdomain
Info: Applying configuration version '1592925160'
Info: Creating state file /opt/puppetlabs/puppet/cache/state/state.yaml
Notice: Applied catalog in 0.02 seconds

```

## Version Control
* Version control softwares allows to track changes on a file over time. Puppet defines your 'Infra as code', thus puppet code can also be used with version control for scalable infra.
* __Git__ is a version control tool, it doesn't tracks files but the changes on them, whenever a new _change is commited_ to the git repo, it only records the changes not the whole file. This makes it much faster and less space consuming
* Git maintains a tree datastructure to track the changes. The root is called __master__ and the folling are called __branches__, 
    * in puppet the master is renamed with __Production__ to avoid confusion with puppet master 
    * __Branches__ are called __dev__ that are used for pre-production
    * the repository is called __Control repo__
* The __R10k__ tool is responsible for pointing, fetching and commiting to-and-fro the control repo.  

## Setup Control Repo

* Step 1: Prepare github 
    * Create a new repo called `control_repo`
    * make it public and initialize readme
    * Create a new branch called `production` from the branch drop-down 
    * make `production` as the default branch from settings 
    * Goto to code --> branches and remove `master` branch
* Step 2: Connect github with R10k
    * Create a directory: `mkdir /etc/puppetlabs/r10k`
    * Create r10k YML config: `nano /etc/puppetlabs/r10k/r10k.yaml`
    
```
---
:cachedir: '/var/cache/r10k'

:sources:
    :my-org:
                remote: 'https://github.com/rishiCSE17/control_repo.git'
                basedir: '/etc/puppetlabs/code/environments'
```

* Step 3: `r10k deploy environment -p` to execute r10k

# Puppet building blocks

## Resources 
### File Resource
* `'/root/README'` : full path of the file
* `ensure => file` : validate the type is file
* `content => 'hello world` : content that must be 
* `,`: trailing comma -- finish confing by a trailing comma (best practice)
```
file{ '/root/README':
    ensure => file,
    content => 'hello world',
}
```

### service Resource 
* `'cron'`: name of the service
* `ensure => running` : state of the service 
* `enable => true,` : start at boot
```
service{ 'crond':
    ensure => running, 
    enable => true,
}
```
Puppet doesn't allow to declair same resource for the same node... i.e. if another file block is declaired for `/root/README` it won't work. 

## Manage a file in site.pp

* crete a new file in github as `control_repo/manifest/site.pp`, by creating a directory `manifest`first.    

```|
node default {
  file{'/root/README':
    ensure => file,
    content => 'this is a test file',
  }
}
```

* `r10k deploy environment -p` : Fetches the content from the github repo
* `puppet agent -t` : Applies changes to the system 

```
Info: Using configured environment 'production'
Info: Retrieving pluginfacts
Info: Retrieving plugin
Info: Retrieving locales
Info: Caching catalog for localhost.localdomain
Info: Applying configuration version '1592932721'
Notice: /Stage[main]/Main/Node[default]/File[/root/README]/content: 
--- /root/README	2020-06-23 10:13:31.715339896 -0700
+++ /tmp/puppet-file20200623-6241-1vs0bm9	2020-06-23 10:18:41.477012267 -0700
@@ -0,0 +1 @@
+this is a test file
\ No newline at end of file

Info: Computing checksum on file /root/README
Info: /Stage[main]/Main/Node[default]/File[/root/README]: Filebucketed /root/README to puppet with sum d41d8cd98f00b204e9800998ecf8427e
Notice: /Stage[main]/Main/Node[default]/File[/root/README]/content: content changed '{md5}d41d8cd98f00b204e9800998ecf8427e' to '{md5}a5890ace30a3e84d9118196c161aeec2'
Notice: Applied catalog in 0.11 seconds
[root@localhost ~]# 
```

## Class
A collection of several resources. The process of adding called clasification.
